In [2]:
print("Hello World")

Hello World


# **Overall Goal**
We are building a data pipeline for analyzing popular Python repositories from GitHub, using their public API. The pipeline:
- Fetches repository data (authenticated via token)
- Cleans and standardizes the data
- Performs data transformations and feature engineering
- Filters and segments data based on business rules

# **All the imports that ere needed**

In [3]:
import requests
import pandas as pd
from datetime import datetime

# **GitHub Authentication Setup**

In [ ]:
GITHUB_TOKEN = "YOUR_GITHUB_TOKEN_HERE"

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

# **GitHub API Call with Exception Handling**

In [5]:
url = "https://api.github.com/search/repositories"
params = {
    "q": "language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 50
}
try:
    response = requests.get(url, headers=HEADERS, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data:
        print("Data fetched successfully")

except requests.exceptions.Timeout:
    raise Exception("Request timed out")

except requests.exceptions.HTTPError as e:
    raise Exception(f"HTTP error occurred: {e}")

except requests.exceptions.RequestException as e:
    raise Exception(f"Request failed: {e}")

Data fetched successfully


# **Extracting Repository Items**

In [6]:
items = data.get("items", [])
len(items)

50

# **Normalizing JSON into the DataFrame**

In [7]:
df = pd.json_normalize(items)
df.head()

,id,node_id,name,full_name,private,html_url,description,fork,url,forks_url,...,license.name,license.spdx_id,license.url,license.node_id,permissions.admin,permissions.maintain,permissions.push,permissions.triage,permissions.pull,license
0,54346799,MDEwOlJlcG9zaXRvcnk1NDM0Njc5OQ==,public-apis,public-apis/public-apis,False,https://github.com/public-apis/public-apis,A collective list of free APIs,False,https://api.github.com/repos/public-apis/publi...,https://api.github.com/repos/public-apis/publi...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN
1,13491895,MDEwOlJlcG9zaXRvcnkxMzQ5MTg5NQ==,free-programming-books,EbookFoundation/free-programming-books,False,https://github.com/EbookFoundation/free-progra...,:books: Freely available programming books,False,https://api.github.com/repos/EbookFoundation/f...,https://api.github.com/repos/EbookFoundation/f...,...,Creative Commons Attribution 4.0 International,CC-BY-4.0,https://api.github.com/licenses/cc-by-4.0,MDc6TGljZW5zZTI1,False,False,False,False,True,NaN
2,83222441,MDEwOlJlcG9zaXRvcnk4MzIyMjQ0MQ==,system-design-primer,donnemartin/system-design-primer,False,https://github.com/donnemartin/system-design-p...,Learn how to design large-scale systems. Prep ...,False,https://api.github.com/repos/donnemartin/syste...,https://api.github.com/repos/donnemartin/syste...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
3,21289110,MDEwOlJlcG9zaXRvcnkyMTI4OTExMA==,awesome-python,vinta/awesome-python,False,https://github.com/vinta/awesome-python,An opinionated list of awesome Python framewor...,False,https://api.github.com/repos/vinta/awesome-python,https://api.github.com/repos/vinta/awesome-pyt...,...,Other,NOASSERTION,None,MDc6TGljZW5zZTA=,False,False,False,False,True,NaN
4,63476337,MDEwOlJlcG9zaXRvcnk2MzQ3NjMzNw==,Python,TheAlgorithms/Python,False,https://github.com/TheAlgorithms/Python,All Algorithms implemented in Python,False,https://api.github.com/repos/TheAlgorithms/Python,https://api.github.com/repos/TheAlgorithms/Pyt...,...,MIT License,MIT,https://api.github.com/licenses/mit,MDc6TGljZW5zZTEz,False,False,False,False,True,NaN


# **Checking for NaN enteries**

In [8]:
df.isna().sum()

id                       0
node_id                  0
name                     0
full_name                0
private                  0
                        ..
permissions.maintain     0
permissions.push         0
permissions.triage       0
permissions.pull         0
license                 50
Length: 108, dtype: int64

# **Checking for Data information**

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Columns: 108 entries, id to license
dtypes: bool(19), float64(2), int64(10), object(77)
memory usage: 35.8+ KB


# **Selecting the Relevant Columns only**

In [10]:
columns = [
    "name",
    "full_name",
    "owner.login",
    "stargazers_count",
    "forks_count",
    "open_issues_count",
    "language",
    "created_at",
    "updated_at",
    "license.name",
    "html_url"
]

df = df[columns]
df.head()


,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20T23:49:42Z,2025-12-16T10:38:21Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11T06:50:37Z,2025-12-16T10:34:29Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26T16:15:28Z,2025-12-16T10:38:53Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27T21:00:06Z,2025-12-16T10:37:03Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16T09:44:01Z,2025-12-16T10:35:37Z,MIT License,https://github.com/TheAlgorithms/Python


# Data Cleaning

# **Removing repositories with missing language**

In [11]:
df_clean = df.dropna(subset=["language"])
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20T23:49:42Z,2025-12-16T10:38:21Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11T06:50:37Z,2025-12-16T10:34:29Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26T16:15:28Z,2025-12-16T10:38:53Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27T21:00:06Z,2025-12-16T10:37:03Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16T09:44:01Z,2025-12-16T10:35:37Z,MIT License,https://github.com/TheAlgorithms/Python


# **Filling missing license with "No License"**

In [12]:
df_clean["license.name"] = df_clean["license.name"].fillna("No License")
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20T23:49:42Z,2025-12-16T10:38:21Z,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11T06:50:37Z,2025-12-16T10:34:29Z,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26T16:15:28Z,2025-12-16T10:38:53Z,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27T21:00:06Z,2025-12-16T10:37:03Z,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16T09:44:01Z,2025-12-16T10:35:37Z,MIT License,https://github.com/TheAlgorithms/Python


# **Data Type Conversion dateTime only**

In [13]:
df_clean["created_at"] = pd.to_datetime(df_clean["created_at"])
df_clean["updated_at"] = pd.to_datetime(df_clean["updated_at"])
df_clean.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python


# Feature Engineering

# **Adding Repository age (days) & Stars-to-forks ratio**

In [14]:
today = pd.Timestamp.utcnow()

df_clean["repo_age_days"] = (today - df_clean["created_at"]).dt.days

df_clean["stars_forks_ratio"] = (
    df_clean["stargazers_count"] /
    df_clean["forks_count"].replace(0, 1)
)

df_clean.head()


,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847


# **Adding filters to get good data (stars, repo age, owners)**

In [15]:
df_filtered = df_clean[
    (df_clean["stargazers_count"] > 10_000) &
    (df_clean["repo_age_days"] > 365) &
    (df_clean["owner.login"].str.len() > 2)
]

df_filtered.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847


# **Rechecking of Null enteries after all processing**

In [16]:
print(df_filtered.shape)
print(df_filtered.isnull().sum())

(49, 13)
name                 0
full_name            0
owner.login          0
stargazers_count     0
forks_count          0
open_issues_count    0
language             0
created_at           0
updated_at           0
license.name         0
html_url             0
repo_age_days        0
stars_forks_ratio    0
dtype: int64


# **Saving the most updated & cleaned data**

In [17]:
output_file = "cleaned_popular_python_repos.csv"
df_filtered.to_csv(output_file, index=False)
output_file


'cleaned_popular_python_repos.csv'

# Dividing the data into differnet CSVs

# **Applying filters to Search Repositories by Owner Name**

In [18]:
org_repos = df_clean[
    df_clean["owner.login"].str.contains("-", na=False)
]

org_repos.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
5,AutoGPT,Significant-Gravitas/AutoGPT,Significant-Gravitas,180318,46181,315,Python,2023-03-16 09:21:07+00:00,2025-12-16 09:55:46+00:00,Other,https://github.com/Significant-Gravitas/AutoGPT,1006,3.904593
8,langflow,langflow-ai/langflow,langflow-ai,141277,8172,935,Python,2023-02-08 22:28:03+00:00,2025-12-16 10:39:47+00:00,MIT License,https://github.com/langflow-ai/langflow,1041,17.287934
9,youtube-dl,ytdl-org/youtube-dl,ytdl-org,139181,10575,4120,Python,2010-10-31 14:35:07+00:00,2025-12-16 10:18:28+00:00,The Unlicense,https://github.com/ytdl-org/youtube-dl,5524,13.161324
10,yt-dlp,yt-dlp/yt-dlp,yt-dlp,138431,11178,2224,Python,2020-10-26 04:22:55+00:00,2025-12-16 10:38:16+00:00,The Unlicense,https://github.com/yt-dlp/yt-dlp,1877,12.384237


# **Applying filteres to Search by License Type**

In [19]:
licensed_repos = df_clean[
    df_clean["license.name"].isin(["MIT License", "Apache License 2.0"])
]

licensed_repos.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847
7,transformers,huggingface/transformers,huggingface,153922,31438,2163,Python,2018-10-29 13:56:00+00:00,2025-12-16 10:38:16+00:00,Apache License 2.0,https://github.com/huggingface/transformers,2604,4.896049
8,langflow,langflow-ai/langflow,langflow-ai,141277,8172,935,Python,2023-02-08 22:28:03+00:00,2025-12-16 10:39:47+00:00,MIT License,https://github.com/langflow-ai/langflow,1041,17.287934
12,langchain,langchain-ai/langchain,langchain-ai,122027,20129,286,Python,2022-10-17 02:58:36+00:00,2025-12-16 10:33:47+00:00,MIT License,https://github.com/langchain-ai/langchain,1156,6.062248


# **Applying Filter by Popularity Buckets**

In [20]:
def popularity_bucket(stars):
    if stars >= 50000:
        return "Very High"
    elif stars >= 20000:
        return "High"
    elif stars >= 10000:
        return "Medium"
    else:
        return "Low"

df_clean["popularity_level"] = df_clean["stargazers_count"].apply(popularity_bucket)

df_clean["popularity_level"].value_counts()


popularity_level
Very High    50
Name: count, dtype: int64

# **Applying Filters by Popularity Level**

In [21]:
high_popularity = df_clean[
    df_clean["popularity_level"].isin(["High", "Very High"])
]

high_popularity.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio,popularity_level
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432,Very High
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080,Very High
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375,Very High
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436,Very High
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847,Very High


# **Applying filters to get Recently Updated Repositories**

In [22]:
recent_repos = df_clean[
    (pd.Timestamp.utcnow() - df_clean["updated_at"]).dt.days <= 90
]

recent_repos.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio,popularity_level
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432,Very High
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080,Very High
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375,Very High
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436,Very High
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847,Very High


# **Applying Filters by Stars-to-Forks Efficiency**

In [23]:
efficient_repos = df_clean[
    df_clean["stars_forks_ratio"] >= 10
]

efficient_repos.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio,popularity_level
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436,Very High
8,langflow,langflow-ai/langflow,langflow-ai,141277,8172,935,Python,2023-02-08 22:28:03+00:00,2025-12-16 10:39:47+00:00,MIT License,https://github.com/langflow-ai/langflow,1041,17.287934,Very High
9,youtube-dl,ytdl-org/youtube-dl,ytdl-org,139181,10575,4120,Python,2010-10-31 14:35:07+00:00,2025-12-16 10:18:28+00:00,The Unlicense,https://github.com/ytdl-org/youtube-dl,5524,13.161324,Very High
10,yt-dlp,yt-dlp/yt-dlp,yt-dlp,138431,11178,2224,Python,2020-10-26 04:22:55+00:00,2025-12-16 10:38:16+00:00,The Unlicense,https://github.com/yt-dlp/yt-dlp,1877,12.384237,Very High
11,HelloGitHub,521xueweihan/HelloGitHub,521xueweihan,137391,10995,229,Python,2016-05-04 06:24:11+00:00,2025-12-16 10:22:29+00:00,No License,https://github.com/521xueweihan/HelloGitHub,3513,12.495771,Very High


# **Saving Organization Repositories only**

In [24]:
org_repos.to_csv("github_org_repos.csv", index=False)

# **Save Licensed Repositories only**

In [25]:
licensed_repos.to_csv("github_licensed_repos.csv", index=False)

# **Saving High Popularity Repositories only**

In [26]:
high_popularity.to_csv("github_high_popularity_repos.csv", index=False)

# **Saving Recently Updated Repositories only**

In [27]:
recent_repos.to_csv("github_recent_repos.csv", index=False)

# **Saving High Engagement Repositories only**

In [28]:
efficient_repos.to_csv("github_high_engagement_repos.csv", index=False)

# **Imports fro DB Connection**

In [29]:
!pip install -q pandas sqlalchemy pymysql

# **DB Connection String**

In [30]:
import pandas as pd
from sqlalchemy import create_engine

DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "python_ca2"
DB_USER = "root"
DB_PASS = ""

engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
    pool_pre_ping=True
)

with engine.connect() as conn:
    print("Connected to MySQL via XAMPP!")

Connected to MySQL via XAMPP!


In [ ]:
csv_path = "cleaned_popular_python_repos.csv"
df = pd.read_csv(csv_path)
df.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847


In [32]:
table_name = "cleaned_popular_python_repos"

df.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False,
    chunksize=2000
)

print(f"Saved {len(df)} rows into `{table_name}`")

Saved 49 rows into `cleaned_popular_python_repos`


In [33]:
df_from_db = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 50", con=engine)
df_from_db.head()

,name,full_name,owner.login,stargazers_count,forks_count,open_issues_count,language,created_at,updated_at,license.name,html_url,repo_age_days,stars_forks_ratio
0,public-apis,public-apis/public-apis,public-apis,385168,41109,755,Python,2016-03-20 23:49:42+00:00,2025-12-16 10:38:21+00:00,MIT License,https://github.com/public-apis/public-apis,3557,9.369432
1,free-programming-books,EbookFoundation/free-programming-books,EbookFoundation,378893,65631,204,Python,2013-10-11 06:50:37+00:00,2025-12-16 10:34:29+00:00,Creative Commons Attribution 4.0 International,https://github.com/EbookFoundation/free-progra...,4449,5.773080
2,system-design-primer,donnemartin/system-design-primer,donnemartin,329477,53640,516,Python,2017-02-26 16:15:28+00:00,2025-12-16 10:38:53+00:00,Other,https://github.com/donnemartin/system-design-p...,3214,6.142375
3,awesome-python,vinta/awesome-python,vinta,273989,26908,526,Python,2014-06-27 21:00:06+00:00,2025-12-16 10:37:03+00:00,Other,https://github.com/vinta/awesome-python,4189,10.182436
4,Python,TheAlgorithms/Python,TheAlgorithms,214847,49620,801,Python,2016-07-16 09:44:01+00:00,2025-12-16 10:35:37+00:00,MIT License,https://github.com/TheAlgorithms/Python,3440,4.329847
